<a href="https://colab.research.google.com/github/coulibalymoumouni/Python_seance/blob/main/Exercice_prediction_avec_calculs_des_metriques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importer les données
import pandas as pd
diabete = pd.read_excel('/content/sample_data/diabete.xlsx')
diabete.head()

,pregnant,plasma,diastolic,triceps,serum,bodymass,pedigree,age,alea1,alea2,diabete
0,0,138,0,0,0,36.3,0.933,25,0.338,0.188,positive
1,4,142,86,0,0,44.0,0.645,22,0.835,0.711,positive
2,3,142,80,15,0,32.4,0.200,63,0.493,0.845,negative
3,3,113,50,10,85,29.5,0.626,25,0.857,0.821,negative
4,5,88,78,30,0,27.6,0.258,37,0.045,0.392,negative


In [2]:
#recodage de la variable diabete
recodage = {'positive': 1, 'negative': 0}

#creation de la variable cible
diabete['cible'] = diabete['diabete'].map(recodage)
diabete.head()



,pregnant,plasma,diastolic,triceps,serum,bodymass,pedigree,age,alea1,alea2,diabete,cible
0,0,138,0,0,0,36.3,0.933,25,0.338,0.188,positive,1
1,4,142,86,0,0,44.0,0.645,22,0.835,0.711,positive,1
2,3,142,80,15,0,32.4,0.200,63,0.493,0.845,negative,0
3,3,113,50,10,85,29.5,0.626,25,0.857,0.821,negative,0
4,5,88,78,30,0,27.6,0.258,37,0.045,0.392,negative,0


In [6]:
#suppression de la variable cible et diabete
#pour garder les variables explicatives
x = diabete.drop(['diabete','cible'], axis=1)

#mettre la variable cible dans y
y = diabete['cible']
x.head()


,pregnant,plasma,diastolic,triceps,serum,bodymass,pedigree,age,alea1,alea2
0,0,138,0,0,0,36.3,0.933,25,0.338,0.188
1,4,142,86,0,0,44.0,0.645,22,0.835,0.711
2,3,142,80,15,0,32.4,0.200,63,0.493,0.845
3,3,113,50,10,85,29.5,0.626,25,0.857,0.821
4,5,88,78,30,0,27.6,0.258,37,0.045,0.392


In [7]:
#on crée 2 echantillons de test avec 20% pour l'échantillon de test
#les échantillons sont effectués selon un tirage aléatoire
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print("La base globale", x.shape)
print("La base d'apprentissage", X_train.shape)
print("La base de test", X_test.shape)


La base globale (568, 10)
La base d'apprentissage (454, 10)
La base de test (114, 10)


In [8]:
#importation de la bibliotheque pour la regression logistique
from sklearn.linear_model import LogisticRegression

#calibrage de notre regression logistique
model = LogisticRegression(solver='liblinear')

#entrainement du modèle
result = model.fit(X_train, y_train)

print(result.coef_,result.intercept_)




[[ 1.02038842e-01  2.42846004e-02 -2.20800610e-02  8.50058490e-03
  -1.96407290e-04  3.33851084e-02  4.48116086e-01  1.48390400e-02
  -4.22229149e-01  4.09996683e-02]] [-4.3129321]


In [9]:
X_train.columns

Index(['pregnant', 'plasma', 'diastolic', 'triceps', 'serum', 'bodymass',
       'pedigree', 'age', 'alea1', 'alea2'],
      dtype='object')

In [11]:
#application du modele sur l'echantillon test
y_pred = result.predict(X_test)

#afficher les résultats de la prédiction
print(y_pred)

[0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1
 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0
 0 1 1]


In [13]:
#transformation de y_pred en dataframe
df_pred = pd.DataFrame(y_pred)
df_pred.head()

#exportation en fichier excel
df_pred.to_excel('Mespredictions.xlsx')

In [14]:
#calcul de la matrice de confusion
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

print(confusion_matrix(y_test, y_pred))

[[60  8]
 [27 19]]


In [15]:
#on stocke la matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[60  8]
 [27 19]]


In [16]:
#on stock les valeurs de la matrice de confusion
TP = cm[0,0]
TN = cm[1,1]
FP = cm[1,0]
FN = cm[0,1]
print(TP,TN,FP,FN)

60 19 27 8


In [17]:
accuracy = (TP+TN)/(TP+TN+FP+FN)
print(accuracy)


0.6929824561403509


In [18]:
#on va utiliser la méthode RFE de selection des variables
#on va garder uniquemet les variables les plus explicatives
#et qui sont significatives
from sklearn.feature_selection import RFE
selecteur = RFE (estimator=model)

#on stock dans sol le modele avec uniquement les variables explicatives
sol = selecteur.fit(X_train, y_train)
print("Le nombre de variables retenues",sol.n_features_)
print("Les variables retenues",sol.support_)
print("Classement des variables retenues",sol.ranking_)


Le nombre de variables retenues 5
Les variables retenues [ True False False False False  True  True False  True  True]
Classement des variables retenues [1 2 3 5 6 1 1 4 1 1]


In [19]:
X_train.columns

Index(['pregnant', 'plasma', 'diastolic', 'triceps', 'serum', 'bodymass',
       'pedigree', 'age', 'alea1', 'alea2'],
      dtype='object')

In [20]:
#on stock dans sol_all le modele avec uniquement les variables explicatives
#sur toute la base
sol_all = selecteur.fit(x, y)
print("Le nombre de variables retenues",sol_all.n_features_)
print("Les variables retenues",sol_all.support_)
print("Classement des variables retenues",sol_all.ranking_)

Le nombre de variables retenues 5
Les variables retenues [ True False False False False  True  True False  True  True]
Classement des variables retenues [1 2 3 5 6 1 1 4 1 1]


In [21]:

#supprimer les colonnes non explicatives
X_new_train = X_train.loc[:,sol.support_]
print(X_new_train.head())


     pregnant  bodymass  pedigree  alea1  alea2
524         8      23.6     0.840  0.790  0.855
550         4      29.7     0.361  0.480  0.458
456         7      28.8     0.687  0.306  0.061
320         4      38.5     0.554  0.021  0.036
45          0      43.5     0.347  0.874  0.402


In [23]:
print('la base globale',x.shape)
print('la nouvelle base d apprentissage',X_new_train.shape)

la base globale (568, 10)
la nouvelle base d apprentissage (454, 5)


In [24]:
#estimation du model final
model_final = LogisticRegression(solver='liblinear')
result_new = model_final.fit(X_new_train, y_train)
print(result_new.coef_,result_new.intercept_)

[[ 0.13699415  0.05127268  0.61761537 -0.42241441  0.21007885]] [-3.07486921]


In [25]:
X_new_train.columns

Index(['pregnant', 'bodymass', 'pedigree', 'alea1', 'alea2'], dtype='object')

In [26]:
#estimer mes nouvelles prévisions en fonction du modele
#final
y_pred_new = result_new.predict(X_new_train)
print(y_pred_new)

[0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0
 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0]


In [29]:
#transformer dans un dataframe
df_pred_new = pd.DataFrame(y_pred_new)
print(df_pred_new)

     0
0    0
1    0
2    0
3    0
4    0
..  ..
449  1
450  0
451  0
452  0
453  0

[454 rows x 1 columns]


In [30]:
#exporter les resultats final sos format xlsx
df_pred_new.to_excel('Mespredictions_finales.xlsx')

In [31]:
#calcul de la matrice de confusion
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred_new)

print(confusion_matrix(y_train, y_pred_new))

[[277  27]
 [114  36]]


In [32]:
#stocker la matrice selectionnée
cm_new = confusion_matrix(y_train, y_pred_new)
print(cm_new)

[[277  27]
 [114  36]]


In [33]:
#stocker les valeurs de la matrice
TP_new = cm_new[0,0]
TN_new = cm_new[1,1]
FP_new = cm_new[1,0]
FN_new = cm_new[0,1]
print(TP_new,TN_new,FP_new,FN_new)

277 36 114 27


In [34]:
#calcul de l'accuracy
accuracy_new = (TP_new+TN_new)/(TP_new+TN_new+FP_new+FN_new)
print(accuracy_new)

0.6894273127753304


In [35]:
TN_new, FP_new, FN_new, TP_new = cm_new.ravel()

print(f"True Negatives (TN): {TN_new}")
print(f"False Positives (FP): {FP_new}")
print(f"False Negatives (FN): {FN_new}")
print(f"True Positives (TP): {TP_new}")


True Negatives (TN): 277
False Positives (FP): 27
False Negatives (FN): 114
True Positives (TP): 36


In [44]:
#Indique la proportion de prédictions correctes parmi toutes les prédictions
accuracy = (TP_new+TN_new)/(TP_new+TN_new+FP_new+FN_new)
print(accuracy.round(2))

0.69


In [43]:
#Mesure la proportion des prédictions positives qui sont réellement positives. Utile lorsque les faux positifs (FP) doivent être minimisés.
precision = TP_new/(TP_new+FP_new)
print(precision.round(2))

0.57


In [41]:
#Mesure la capacité du modèle à détecter les classes positives. Utile lorsque les faux négatifs (FN) doivent être minimisés.
recall = TP_new/(TP_new+FN_new)
print(recall.round(2))

0.24


In [45]:
#Moyenne harmonique entre précision et rappel. C'est utile lorsque vous voulez un équilibre entre précision et rappel.
f1_score = 2 * (precision * recall) / (precision + recall)
print(f1_score.round(2))

0.34


In [47]:
#Mesure la capacité du modèle à détecter les classes négatives.
specificity = TN_new/(TN_new+FP_new)
print(specificity.round(2))

0.91


In [48]:
#ROC-AUC (Receiver Operating Characteristic - Area Under Curve)
#Représente la performance globale du modèle en mesurant la discrimination entre les classes.
#Plus le score est proche de 1, meilleur est le modèle
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y_train, y_pred_new)
print(auc_score.round(2))

0.58


In [51]:
#taux d'erreur : indique la proportion des prédictions incorrectes
taux_erreur = 1 - accuracy
print(taux_erreur.round(2))

0.31


In [53]:
taux_erreur_new = (FP_new + FN_new)/ (TP_new+TN_new+FP_new+FN_new)
print(taux_erreur_new.round(2))

0.31
